In [ ]:
import sys
import os
import time

In [ ]:
# include path to our library
cwd = os.getcwd()
dir_path = os.path.dirname(os.path.realpath(cwd))
sys.path.append(dir_path)
time.sleep(2)

In [ ]:
dir_path
%cd $dir_path
!ls

In [ ]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
from core.data.data_loader import *
from sklearn.model_selection import train_test_split

In [ ]:
prefix = './dataset/2020_100_sorted.csv'
prefix = './dataset/phi025-025_eta025-025_train1_lasthit_20200219.csv'

HITS = 10

data = Dataset(prefix, KindNormalization.Zscore)
#x, y = data.get_training_data(normalise=False, cylindrical=False, hit=10)
dataset = data.get_training_data(cylindrical=False, hits=HITS)
#X, y = data.convert_to_supervised(x.values, n_hit_in=4, n_hit_out=1,n_features=3)
dataset = dataset.iloc[0:2640,0:]
print(dataset.shape)

In [ ]:
#dataset

In [ ]:
def convert_to_supervisedd(sequences, n_hit_in, n_hit_out, n_features):
    '''
        n_hit_in : 3 number of hits
        n_hit_out: 1 number of future hits
        n_features 3
    '''
    X , Y = [],[]

    rows = sequences.shape[0]
    cols = sequences.shape[1]
 

    for i in range(0, rows):
        print('begin track ', i)
        
        end_idx = 0
        out_end_idx = 0
        
        for j in range(0, cols, n_features):
            end_idx = j + n_hit_in*n_features
            out_end_idx = end_idx + n_hit_out*n_features

            if out_end_idx > cols+1:
                print('\t cols %s, end_ix %s , out_end_idx %s' % ( cols, end_idx, out_end_idx))
                print('end track  ', i) 
                break
            
            print('\t x[%s,%s:%s] y[%s,%s:%s]' % (i, j, end_idx, i, end_idx, out_end_idx))            
            seq_x, seq_y = sequences[i, j:end_idx], sequences[i, end_idx:out_end_idx]
            
            X.append(seq_x)
            Y.append(seq_y)
            
    print(rows, cols)
    #return np.array(X) , np.array(Y)
    return pd.DataFrame(X) , pd.DataFrame(Y)   



In [ ]:
dataset.to_csv('verifica.csv')

In [ ]:
Normalise = True
X, y = data.convert_to_supervised(dataset.values, n_hit_in=4, n_hit_out=1,n_features=3, normalise=Normalise)
# there is empty data?
print(X.isnull().values.any())
print(y.isnull().values.any())
print(X.shape)


In [ ]:
# concat to see if they are correct
#pd.concat([X, y], axis =1)

In [ ]:
p = 0.2
isplit = round(len(dataset)*p)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=p, shuffle=False)
print('[Data] split ', isplit)
print('[Data] shape data dataset.shape:', dataset.shape)
print('[Data] shape data X_train.shape:', X_train.shape)
print('[Data] shape data y_train.shape:', y_train.shape)
print('[Data] shape data X_test.shape:', X_test.shape)
print('[Data] shape data y_test.shape:', y_test.shape)

In [ ]:
# divido dos dados origns
X_train_new, X_test_new = train_test_split(dataset, test_size=0.2, shuffle=False)
print('[Data] shape data dataset.shape:', dataset.shape)
print('[Data] shape data X_train.shape:', X_train_new.shape)
print('[Data] shape data X_test_new.shape:', X_test_new.shape)


In [ ]:
x_true = X_test_new.iloc[:,0:12] #pd.DataFrame(
y_true = X_test_new.iloc[:,12:30]

print('[Data] shape data x_true.shape:', x_true.shape)
print('[Data] shape data y_true.shape:', y_true.shape)

df_test = np.concatenate([x_true, y_true], axis=1)
print('[Data] shape data df_test.shape:' , df_test.shape)
df_test = pd.DataFrame(df_test)
print(df_test.isnull().values.any())
df_test.tail()

In [ ]:
	def convert_supervised_to_normal1(sequences, n_hit_in, n_hit_out, hits):
		'''
			This function convert the predicted sequences to a vector
			n_hit_in : 4 number of hits
			n_hit_out: 1 number of future hits
			hits	 : 10
		'''

		Y = []

		len_pred_seq = hits - n_hit_in
		len_seq = len(sequences)
		end_ix = 0

		for x in range(0, len_seq, len_pred_seq):
			cols = []
			end_ix = x + len_pred_seq
			pred_seq = sequences[x:end_ix,:]

			#print('matrix: ', pred_seq)
					
			pred_seq = np.matrix(pred_seq).flatten().tolist()			
			#result = m_pred_seq.flatten().tolist()
			#print('result: ', pred_seq)
			#print('result 0: ', pred_seq[0])
			#for i in range(0, len(pred_seq)):		
			#	print('\t row added:', pred_seq[i,:])
			#	ocls.append(pred_seq[i,:])
			#print(lst.to_matrix())

			Y.append(pred_seq[0])

		return Y

In [ ]:
conv = convert_supervised_to_normal1(y_test.values, n_hit_in=4, n_hit_out=1, hits=10)
conv = pd.DataFrame(conv)
print(conv.isnull().values.any())
print('[Data] shape data conv.shape:' , conv.shape)
#conv.loc[conv.index.dropna()]
#conv.index.dropna()
#conv.index.dropna()
# get index with nan
inds = pd.isnull(conv).any(1).to_numpy().nonzero()[0]
inds
#conv.index.dropna(inds)
#conv.drop(conv[conv['index'].isnull()].index, inplace=True)
#conv[conv.index.notnull()]

In [ ]:

if Normalise:
    y_test_inv = pd.DataFrame(data.inverse_transform_y(y_test))
else:
    y_test_inv = y_test

y_test_orig = pd.DataFrame(convert_supervised_to_normal1(y_test_inv.values,
                                                        n_hit_in=4, n_hit_out=1, hits=10))

print(y_test_orig.isnull().values.any())
#print(y_test_orig.isnull().sum())

print('[Data] shape data y_test.shape:' , y_test.shape)

if y_test_orig.isnull().values.any():
    y_test_orig = y_test_orig.dropna()

print('[Data] shape data y_test_orig.shape:' , y_test_orig.shape)
print('[Data] shape data x_true.shape:' , x_true.shape)

#if x_true.count() != y_test_orig.count():
#x_true.drop(index=529, inplace=True)
#x_true = x_true.iloc[0:529,:]
#y_true = y_true.iloc[0:529,:]
#print(x_true.shape)
#print(y_test_orig.shape)    
df_recons = np.concatenate([x_true, y_test_orig], axis=1)
df_recons = pd.DataFrame(df_recons)
print('[Data] shape data df_recons.shape:' , df_recons.shape)

In [ ]:
# deve ser iguales
y_test_orig.values == y_true.values
#print(y_true.shape)
#print(y_test_orig.shape)  

In [ ]:
df_test.values == df_recons.values

In [ ]:
#y_test_orig.head()
df_recons.head()

In [ ]:
from core.utils.utils import *

N_tracks = 50
fig = track_plot_xyz([dataset], n_hits = 10, cylindrical = False, n_tracks = 100, 
               path='track-test.html', title='Data original 10 hits')

fig.show()

In [ ]:

fig = track_plot_xyz([df_test, df_recons], n_hits = 10, cylindrical = False, n_tracks = N_tracks, 
               path='track-test.html', title='Data original 10 hits')

fig.show()

In [ ]:

fig = track_plot_xyz([y_true, y_test_orig ], n_hits = 6, cylindrical = False, n_tracks = N_tracks, 
               path='tmp.html', title='tmp')

fig.show()

In [ ]:
fig = track_plot_xyz([df_recons], n_hits = 10, cylindrical = False, n_tracks = N_tracks, 
               path='track-test.html', title='Data original 10 hits')

fig.show()